In [ ]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
from zipfile import ZipFile
import io
import time
import json
import pandas as pd
from wulpus.helper import zip_to_dataframe

In [ ]:
path = '.\\wulpus\\measurements\\wulpus-2025-09-04_11-23-52.zip'

In [ ]:
df, config = zip_to_dataframe(path)
print(f"columns: {list(df.columns)}")
df.head()

In [ ]:
# Plot measurements as imshow heatmaps stacked by tx_rx_id (one subplot per unique tx_rx_id)
import inspect
from matplotlib import colors as mcolors

fig = None

if 'tx_rx_id' not in df.columns:
    print("Column 'tx_rx_id' not found: plotting all measurements as a single heatmap")
    measurements = df['measurement'].dropna()
    lengths = [len(m) for m in measurements]
    max_len = max(lengths) if lengths else 0
    stack = np.array([
        np.pad(np.asarray(m, dtype=float), (0, max_len - len(m)), constant_values=np.nan)
        for m in measurements
    ])
    
    vmin = np.nanmin(stack) if stack.size else -1
    vmax = np.nanmax(stack) if stack.size else 1
    # Build norm (try SymLogNorm if supported, else Normalize)
    norm = mcolors.SymLogNorm(linthresh=10.0, linscale=0.03, vmin=vmin, vmax=vmax)

    fig, ax = plt.subplots(figsize=(10, 6))
    im = ax.imshow(stack.T, aspect='auto', cmap='viridis', interpolation='hamming', norm=norm)
    fig.colorbar(im, ax=ax, label='ADC digital code')
    ax.set_title('All measurements')
    ax.set_xlabel('Acquisition index (time)')
    ax.set_ylabel('Sample index')
    plt.tight_layout()
    plt.show()

else:
    groups = list(df.groupby('tx_rx_id'))
    n_groups = len(groups)
    if n_groups == 0:
        raise ValueError('No groups found for tx_rx_id')

    # Prepare axes: one row per group, share x axis
    fig, axs = plt.subplots(nrows=n_groups, ncols=1, figsize=(10, 2.2 * n_groups), sharex=True)
    if n_groups == 1:
        axs = [axs]

    # Helper to build norm robustly
    def build_norm(vmin, vmax):
        try:
            SymLog = mcolors.SymLogNorm
            sig = inspect.signature(SymLog)
            desired = {'linthresh': 10.0, 'linscale': 0.03, 'vmin': vmin, 'vmax': vmax}
            kwargs = {k: v for k, v in desired.items() if k in sig.parameters}
            if kwargs:
                return SymLog(**kwargs)
        except Exception:
            pass
        return mcolors.Normalize(vmin=vmin, vmax=vmax)

    for ax, (txrx, group_df) in zip(axs, groups):
        measurements = group_df['measurement'].dropna()
        if measurements.empty:
            ax.text(0.5, 0.5, 'No measurements', ha='center', va='center')
            continue

        lengths = [len(m) for m in measurements]
        max_len = max(lengths) if lengths else 0
        stack = np.array([
            np.pad(np.asarray(m, dtype=float), (0, max_len - len(m)), constant_values=np.nan)
            for m in measurements
        ])

        vmin = np.nanmin(stack) if stack.size else -1
        vmax = np.nanmax(stack) if stack.size else 1
        norm = build_norm(vmin, vmax)

        im = ax.imshow(stack.T, aspect='auto', cmap='viridis', interpolation='hamming', norm=norm)

        # Set title using config if available
        title_extra = ''
        try:
            cfg = config.tx_rx_config[txrx]
            title_extra = f"  TX {cfg.tx_channels} -> RX {cfg.rx_channels}"
        except Exception:
            pass
        ax.set_title(f'tx_rx_id: {txrx} (n={len(group_df)})' + title_extra)

        # X ticks as time labels sampled across acquisitions
        num_ticks = min(8, len(measurements))
        if num_ticks > 1:
            frame_indices = np.linspace(0, len(measurements) - 1, num_ticks, dtype=int)
            idx_vals = measurements.index.values
            if np.issubdtype(idx_vals.dtype, np.integer):
                dt_index = pd.to_datetime(idx_vals, unit='us')
            else:
                dt_index = pd.to_datetime(measurements.index)
            labels = [dt.strftime('%H:%M:%S') for dt in dt_index[frame_indices]]
            ax.set_xticks(frame_indices)
            ax.set_xticklabels(labels, rotation=45)
        else:
            ax.set_xticks([0])
            ax.set_xticklabels(['Single acquisition'])

        ax.set_ylabel('Sample index')
        ax.grid(False)
        fig.colorbar(im, ax=ax, label='ADC digital code')

    axs[-1].set_xlabel('Acquisition index (time)')
    plt.tight_layout()
    plt.show()

In [ ]:
data_sel = np.stack(df['measurement'].to_numpy())
channel_id = 0

plt.figure(figsize=(10, 5))
plot_data = plt.plot(data_sel[0], linestyle='-', marker='o', linewidth=1, markersize=2)
plt.ylim(-2500, 2500)
plt.title(f'Acquisition 0/{data_sel.shape[0]} (Channel {channel_id})')
plt.xlabel('Samples')
plt.ylabel('ADC digital code')
plt.grid()
plt.show()

def visualize(frame):
    plot_data[0].set_ydata(data_sel[frame])
    plt.title(f'Acquisition {frame}/{data_sel.shape[0]-1} (Channel {channel_id})')
    plt.draw()

widgets.interact(visualize, frame=widgets.IntSlider(min=0, max=data_sel.shape[0]-1, step=1, value=0))

In [ ]:
# # Visualize all time frames as a heatmap (intensity plot)
# plt.figure(figsize=(12, 6))
# border=1000
# data2 = data_meas[:145, :]
# img = plt.imshow(data2.T, aspect='auto', cmap='viridis', interpolation='nearest', vmin=-border, vmax=border)
# plt.colorbar(img, label='ADC digital code')
# plt.xlabel('Time')
# plt.ylabel('samples in one measurement')
# plt.title(f'Measurement evolution over time (Channel {channel_id})')
# plt.show()

In [ ]:
# Plot with real time on x-axis from DataFrame index
plt.figure(figsize=(12, 6))
border = 1000

padded = np.array([
            np.pad(np.array(m, dtype=float), (0,
                   400 - len(m)), constant_values=np.nan)
            for m in df['measurement']
        ])
result = pd.concat(
[df.drop(columns=['measurement']), pd.DataFrame(padded, index=df.index)], axis=1)

img = plt.imshow(padded.T,
                 aspect='auto',
                 cmap='viridis',
                 interpolation='hamming',
                 #  vmin=-border,
                 #  vmax=border,
                 norm=colors.SymLogNorm(
                     linthresh=10, linscale=0.03, vmin=-border, vmax=border)
                 )
plt.colorbar(img, label='ADC digital code')
plt.title('Measurement evolution over time')
plt.xlabel('Time (measurement index)')
plt.ylabel('Sample index within measurement (distance of reflection)')

# Set x-ticks to real time from DataFrame index
num_ticks = 10
frame_indices = np.linspace(0, len(df.index)-1, num_ticks, dtype=int)

# Convert index to datetime and format as HH:MM:SS
idx_vals = df.index.values
if np.issubdtype(idx_vals.dtype, np.integer):
    dt_index = pd.to_datetime(idx_vals, unit='us')
else:
    dt_index = pd.to_datetime(df.index)
plt.xticks(frame_indices, pd.Index(dt_index[frame_indices]).strftime('%H:%M:%S'), rotation=45)

plt.tight_layout()
plt.show()